# Assignment: Подробнее о методах кластеризации

Представим, что международное круизное агентство "Carnival Cruise Line" решило себя разрекламировать с помощью баннеров и обратилось для этого к Вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с Вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры - легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях - Foursqare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив с этой страницы.

Нас будет интересовать файл `checkins.dat`. Открыв его, увидим следующую структуру:

```
id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...
```

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки не содержащие координат - они неинформативны для нас:

```
id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...
```

С помощью `pandas` построим DataFrame и убедимся, что все 396634 строк с координатами считаны успешно.

In [8]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

In [118]:
dat = pd.read_csv('data/checkins.csv')
print dat.head()
print dat.shape

       id    user_id  venue_id   latitude   longitude            created_at
0  984222    15824.0    5222.0  38.895112  -77.036366   2012-04-21 17:43:47
1  984234    44652.0    5222.0  33.800745  -84.410520   2012-04-21 17:43:43
2  984291   105054.0    5222.0  45.523452 -122.676207   2012-04-21 17:39:22
3  984318  2146539.0    5222.0  40.764462 -111.904565   2012-04-21 17:35:46
4  984232    93870.0  380645.0  33.448377 -112.074037   2012-04-21 17:38:18
(396634, 6)


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача - хороший повод познакомиться с алгоритмом `MeanShift`, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [sklearn user guide](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте `MeanShift`, указав `bandwidth=0.1`, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

**Примечание**: на 396634 строках, кластеризация будет работать долго. Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке). Быть очень терпеливым не возбраняется - результат от этого только улучшится.

In [119]:
# select first 100 000 rows and coordinates:
dat_1 = dat.iloc[:100000,3:5]
print dat_1.head()
print dat_1.shape

    latitude   longitude
0  38.895112  -77.036366
1  33.800745  -84.410520
2  45.523452 -122.676207
3  40.764462 -111.904565
4  33.448377 -112.074037
(100000, 2)


In [120]:
model_clust = MeanShift(bandwidth=0.1)
model_clust.fit(dat_1)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [140]:
# predictions on all dataset
preds = model_clust.predict(dat.iloc[:,3:5])
preds

array([   5,    7,   30, ...,  238, 1223,   98], dtype=int64)

In [141]:
# cluster centers:
centers = model_clust.cluster_centers_
centr = np.zeros((centers.shape[0], centers.shape[1]+1))
centr[:, :-1] = centers
centr[:, 2] = [i for i in range(len(centr))]
del(centers)
print centr
print centr.shape

[[  4.07177164e+01  -7.39918354e+01   0.00000000e+00]
 [  3.34494381e+01  -1.12002140e+02   1.00000000e+00]
 [  3.34463803e+01  -1.11901888e+02   2.00000000e+00]
 ..., 
 [  3.92190608e+01  -1.21061061e+02   3.22700000e+03]
 [  3.13787916e+01  -9.53213317e+01   3.22800000e+03]
 [  5.07194116e+01  -1.98112960e+00   3.22900000e+03]]
(3230L, 3L)


In [142]:
dat['cluster'] = preds
dat.head()

,id,user_id,venue_id,latitude,longitude,created_at,cluster,cnt_cluster
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47,5,10959
1,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43,7,7159
2,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22,30,2275
3,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46,66,582
4,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18,1,10895


Некоторые из получившихся кластеров содержат слишком мало точек - такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [143]:
# count observations per cluster and remove clusters with <= 15  obs
tmp = dat.groupby('cluster', as_index=False)\
    .count()[['cluster', 'id']]\
    .rename(columns={'id': 'cnt_cluster'})
    
# use merge or suffix!    
# df_a.join(df_b, on='mukey', how='left', lsuffix='_left', rsuffix='_right')
#dat = dat.merge(tmp, how='left', on='cluster')

In [149]:
# select cols and filter rows example
# rows
dat[dat['cnt_cluster'] > 15].head(1)
# cols
dat[dat.columns.difference(['created_at'])].head(1)
dat.query('cnt_cluster > 15')
# centr_1.columns

,cluster,cnt_cluster,id,latitude,longitude,user_id,venue_id
0,5,10959,984222,38.895112,-77.036366,15824.0,5222.0


In [159]:
centr_1[centr_1['cnt_cluster'] > 15].head(1)

,lat,lon,cluster,cnt_cluster
0,40.717716,-73.991835,0,56450


In [162]:
# join to centers cnt_cluster:
centr_1 = pd.DataFrame( centr, 
                      #index = range(len(dat_1)),
                      columns=['lat', 'lon', 'cluster'])
#centr_1 = pd.DataFrame(centr).rename(columns={0:'lat', 1:'lon', 2:'cluster'})
centr_1 = centr_1.merge(tmp, how='left', on = 'cluster')
centr_filtered = centr_1[centr_1['cnt_cluster']>15]
#centr_filtered.head()
centr_filtered.shape

(1417, 4)

При желании увидеть получившиеся результаты на карте, можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт `mapcustomizer.com` имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

```
38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...
```

In [172]:
# out
centr_filtered.iloc[:,0:2].to_csv('data/cluster_centrs.csv', index=False)

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

```
33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)
```

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

**Примечание**: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико, чтобы не повлиять на кластеризацию или сортировку.

In [176]:
offices = pd.DataFrame({'latitude':[33.751277,25.867736, 51.503016, 52.378894, 39.366487, -33.868457], 
                        'longitude': [-118.188740, -80.324116, -0.075479, 4.885084, 117.036146, 151.205134 ], 
                         'city' : ['Los Angeles', 'Miami', 'London', 'Amsterdam', 'Beijing', 'Sydney']})
offices

,city,latitude,longitude
0,Los Angeles,33.751277,-118.188740
1,Miami,25.867736,-80.324116
2,London,51.503016,-0.075479
3,Amsterdam,52.378894,4.885084
4,Beijing,39.366487,117.036146
5,Sydney,-33.868457,151.205134


In [199]:
#np.linalg.norm( a - offices.iloc[:,1:3])
def nearest_point(point, others):
    return np.min( np.array([ np.linalg.norm(point - p) for p in others]))

In [200]:
# test
a = np.array(centr_filtered.iloc[1,0:2])
nearest_point(point=a, others= np.array(offices.iloc[:,1:2]))

103.13375473005401

In [212]:
# for each cluster centr calculate distance to nearest office 
dist = np.zeros(len(centr_filtered))
for i in range(len(centr_filtered)):
    #print centr_filtered.iloc[i,0:2]
    #print np.array(offices.iloc[:,1:2])
    dist[i] = nearest_point( point=np.array(centr_filtered.iloc[i,0:2]), others=np.array(offices.iloc[:,1:2]) )
    #print dist[i]

In [215]:
# add dist column
centr_1 = centr_filtered
centr_1['dist'] = dist
centr_1.head()

,lat,lon,cluster,cnt_cluster,dist
0,40.717716,-73.991835,0,56450,84.693463
1,33.449438,-112.002140,1,10895,103.133755
2,33.446380,-111.901888,2,9175,103.055828
3,41.878244,-87.629843,3,15283,92.886217
4,37.688682,-122.409330,4,15099,113.841602


In [ ]:
# select top 20:
centr_1

Предположим, вы получили итоговые 20 точек в следующем виде:

```
latitude, longitude

2.2,-2.2

3.3,-3.3

1.1,-0.1

1.1,-1.1

…

19.19,-19.19
```

Отсортируйте полученный список пар (широта, долгота) по возрастанию (пары сравниваются сначала по первому элементу, затем по второму):

```
latitude, longitude

1.1,-1.1

1.1,-0.1

2.2,-2.2

3.3,-3.3

…

19.19,-19.19
```

В итоговый файл с ответами выведите по очереди широту и долготу каждой точки (в том же порядке), получив одну строчку из 40 чисел, разделенных пробелами:

`1.1 -1.1 1.1 -0.1 2.2 -2.2 3.3 -3.3 … 19.19 -19.19`